<a href="https://colab.research.google.com/github/keisyashakila/MSCI-151/blob/main/MILP_CW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [281]:
from collections import defaultdict
import os
import csv

Correcting import statements:
- `from collection import dafaultdict` to `from collections import defaultdict`
- Assuming `import css` was intended as `import csv` as `css` is not a standard Python module.

In [282]:
from collections import defaultdict
import os
import csv

In [283]:
!pip install pulp

In [284]:
import pulp as pl

In [285]:
baristas = ["Max", "Jiwa", "Fore", "Donna", "Paul"]
days = list(range(1,7))
blocks = list(range(1,5))
H = 4

days 1-7, due to 7 days a week

In [286]:
cost = {"Max": 50000, "Jiwa": 50000, "Fore": 50000, "Donna": 150000, "Paul": 150000}
etype = {"Max": "P", "Jiwa": "P", "Fore": "P", "Donna": "F", "Paul": "F"}
minWeekly_default = {"Max": 12, "Jiwa": 12, "Fore": 12, "Donna": 36, "Paul": 36}

In [287]:
avail = {("Max", 1): 8, ("Max", 2): 8, ("Max", 3): 8, ("Max", 4): 0, ("Max", 5): 8, ("Max", 6): 4, ("Max", 7): 4, ("Jiwa", 1): 4, ("Jiwa", 2): 4, ("Jiwa", 3): 8, ("Jiwa", 4): 8, ("Jiwa", 5): 0, ("Jiwa", 6): 4, ("Jiwa", 7): 0, ("Fore", 1): 8, ("Fore", 2): 0, ("Fore", 3): 8, ("Fore", 4): 8, ("Fore", 5): 8, ("Fore", 6): 8, ("Fore", 7): 0, ("Donna", 1): 12, ("Donna", 2): 12, ("Donna", 3): 12, ("Donna", 4): 12, ("Donna", 5): 12, ("Donna", 6): 12, ("Donna", 7): 8, ("Paul", 1): 12, ("Paul", 2): 8, ("Paul", 3): 12, ("Paul", 4): 12, ("Paul", 5): 8, ("Paul", 6): 12, ("Paul", 7): 12}

In [288]:
days = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}

In [289]:
blocks = {1: "07:00-11:00", 2: "11:00-15:00", 3: "15:00-19:00", 4: "19:00-23:00"}
block_ids = list(blocks.keys())

In [290]:
def print_schedule(y_vars):
    for d in days:
        print(f"\n{day_names[d]}")
        for b in block_ids:
            assigned = [o for o in baristas if y_vars[o, d, b].varValue == 1]
            print(f"  Block {b} ({blocks[b]}): {', '.join(assigned) if assigned else 'None'}")

mathematical model

In [291]:
model = pl.LpProblem("Barista_Scheduling", pl.LpMinimize)

decision variable ​￼

In [292]:
y = pl.LpVariable.dicts(
    "y",
    ((o, d, b) for o in baristas for d in days for b in blocks),
    cat="Binary"
)

Minimise Cost

In [293]:
model += pl.lpSum(H * cost[o] * y[o, d, b] for o in baristas for d in days for b in blocks)

# **Constraints**

Coverage (one barista per block)

In [294]:
for d in days:
    for b in blocks:
        model += pl.lpSum(y[o, d, b] for o in baristas) >= 1

Per‑day availability (hours)

In [295]:
for o in baristas:
    for d in days:
        model += H * pl.lpSum(y[o, d, b] for b in blocks) <= avail[(o, d)]

Daily block limit

In [296]:
for o in baristas:
    for d in days:
        if etype[o] == "P":  # Part-time
            model += pl.lpSum(y[o, d, b] for b in blocks) <= 2
        if etype[o] == "F":  # Full-time
            model += pl.lpSum(y[o, d, b] for b in blocks) <= 3

Weekly minimum hours

In [297]:
for o in baristas:
    model += H * pl.lpSum(y[o, d, b] for d in days for b in blocks) >= minWeekly_default[o]

Binary variables (the problem)

*   **cat="Binary"** part automatically enforces that each variable is either 0 or 1.

## **Big Picture**

•  Coverage → shop never empty.

•  Per-day Availability → respect daily limits.

•  Daily limits → enforce part/full-time rules.

•  Weekly minimums → guarantee fair hours.

•  Binary variables → clean yes/no assignments.

---
This show each assignment of a barista to a block on a day as yes/no. Then, it will choose which are yes (1) and which are no (0)


# **Solving part**

Solve the Model

In [298]:
model.solve()

total_calculated_cost = 0
print("Individual Barista Costs:")
for o in baristas:
    # Get total hours for the barista
    hours_worked = sum(H * y[o, d, b].varValue for d in days for b in block_ids)

    # Get hourly wage for the barista
    hourly_wage = cost[o]

    # Calculate individual cost
    barista_cost = hours_worked * hourly_wage
    total_calculated_cost += barista_cost
    print(f"  {o}: {hours_worked} hours * {hourly_wage} (hourly wage) = {barista_cost}")

print(f"\nTotal Calculated Cost: {total_calculated_cost}")
print(f"Total Cost from Model Objective: {pl.value(model.objective)}")

if total_calculated_cost == pl.value(model.objective):
    print("The calculated total cost matches the model's objective value.")
else:
    print("There is a discrepancy between the calculated total cost and the model's objective value.")

Individual Barista Costs:
  Max: 12.0 hours * 50000 (hourly wage) = 600000.0
  Jiwa: 12.0 hours * 50000 (hourly wage) = 600000.0
  Fore: 16.0 hours * 50000 (hourly wage) = 800000.0
  Donna: 36.0 hours * 150000 (hourly wage) = 5400000.0
  Paul: 36.0 hours * 150000 (hourly wage) = 5400000.0

Total Calculated Cost: 12800000.0
Total Cost from Model Objective: 12800000.0
The calculated total cost matches the model's objective value.


Fore works for 16 hours due to:
*   cheaper than full-timers
*   all rules are still respecting all contraints.
*   Fore has 8 hours available on most weekdays



Weekly Hours per Barista

In [299]:
print("Weekly Hours per Barista:")
for o in baristas:
    total_weekly_hours = sum(H * y[o, d, b].varValue for d in days for b in block_ids)
    print(f"  {o}: {total_weekly_hours} hours")

Weekly Hours per Barista:
  Max: 12.0 hours
  Jiwa: 12.0 hours
  Fore: 16.0 hours
  Donna: 36.0 hours
  Paul: 36.0 hours


Daily Schedule (Readable)

In [300]:
def print_schedule(y_vars):
    for d_id, d_name in days.items():
        print(f"\n{d_name}")
        for b_id in block_ids:
            assigned = [o for o in baristas if y_vars[o, d_id, b_id].varValue == 1]
            print(f"  Block {b_id} ({blocks[b_id]}): {', '.join(assigned) if assigned else 'None'}")

print_schedule(y)


Monday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Paul
  Block 3 (15:00-19:00): Max
  Block 4 (19:00-23:00): Jiwa

Tuesday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Jiwa
  Block 4 (19:00-23:00): Max

Wednesday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Fore
  Block 3 (15:00-19:00): Fore
  Block 4 (19:00-23:00): Paul

Thursday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Paul
  Block 3 (15:00-19:00): Donna
  Block 4 (19:00-23:00): Paul

Friday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Fore
  Block 3 (15:00-19:00): Fore
  Block 4 (19:00-23:00): Paul

Saturday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Jiwa
  Block 3 (15:00-19:00): Donna
  Block 4 (19:00-23:00): Max

Sunday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Paul
  Block 4 (19:00-23:00): Donna


# **Interpretation**

*   Coverage: Every block (07:00–23:00, Monday–Sunday), at least one barista assigned.
*   Assignments:

↪ Donna: 5 days/week

↪ Paul: 5 days/week

↪ Max: 3 days/week

↪ Jiwa: 3 days/week

↪ Fore: 2 days/week.

# **Sensitivity analysis**

close early

In [301]:
# 1. Start fresh
model = pl.LpProblem("Cafe_Scheduling_CloseEarly", pl.LpMinimize)

# 2. Redefine block_ids
block_ids = [1, 2, 3]   # only 3 blocks now

# 3. Rebuild decision variables
y = pl.LpVariable.dicts(
    "y",
    ((o, d, b) for o in baristas for d in days for b in block_ids),
    cat="Binary"
)

# 4. Objective function (same formula, but only 3 blocks now)
model += pl.lpSum(H * cost[o] * y[o, d, b] for o in baristas for d in days for b in block_ids)

# 5. Rebuild all constraints fresh, using the new block_ids
# Coverage
for d in days:
    for b in block_ids:
        model += pl.lpSum(y[o, d, b] for o in baristas) >= 1

# Availability
for o in baristas:
    for d in days:
        model += H * pl.lpSum(y[o, d, b] for b in block_ids) <= avail[(o, d)]

# Daily block limits
for o in baristas:
    for d in days:
        if etype[o] == "P":
            model += pl.lpSum(y[o, d, b] for b in block_ids) <= 2
        else:
            model += pl.lpSum(y[o, d, b] for b in block_ids) <= 3

# Weekly minimums
for o in baristas:
    model += H * pl.lpSum(y[o, d, b] for d in days for b in block_ids) >= minWeekly_default[o]

In [302]:
model.solve()
print("Status:", pl.LpStatus[model.status])
print("Total Cost:", pl.value(model.objective))

Status: Optimal
Total Cost: 12600000.0


In [303]:
print("\nSchedule with 3 blocks (closing early):")
print_schedule(y)


Schedule with 3 blocks (closing early):

Monday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Donna, Paul
  Block 3 (15:00-19:00): Paul

Tuesday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Jiwa
  Block 3 (15:00-19:00): Paul

Wednesday
  Block 1 (07:00-11:00): Donna, Paul
  Block 2 (11:00-15:00): Max
  Block 3 (15:00-19:00): Max

Thursday
  Block 1 (07:00-11:00): Fore
  Block 2 (11:00-15:00): Jiwa
  Block 3 (15:00-19:00): Fore

Friday
  Block 1 (07:00-11:00): Donna, Paul
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Donna, Paul

Saturday
  Block 1 (07:00-11:00): Max, Jiwa, Paul
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Fore

Sunday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Donna


## Summary:

### Data Analysis Key Findings
*   The PuLP optimization model was successfully reconfigured for a 3-block schedule, reflecting operations until 19:00.
*   A detailed schedule was generated, assigning baristas to three blocks (07:00-11:00, 11:00-15:00, and 15:00-19:00) for each day of the week. For instance, on Monday, the schedule includes Donna in Block 1, Donna and Paul in Block 2, and Paul in Block 3.
*   This new schedule confirms the model's ability to produce staffing assignments for an early closing time, using only the specified three operational blocks.

### Insights or Next Steps
*   Although the schedule was successfully generated, the total optimized cost for this 3-block schedule was not explicitly displayed in the provided output. The next step should be to extract and report this specific value to fully address the task.


Raise part‑time weekly minimums

# Task
Modify the `minWeekly_default` dictionary to reflect the increased weekly minimum hours for part-time baristas (Max, Jiwa, Fore) from 12 to 16 hours, then rebuild the entire PuLP optimization model with the original 4 blocks, incorporating the updated `minWeekly_default`. Finally, solve the newly built model and report the solution status, total optimized cost, and weekly hours per barista.

## update_min_weekly_hours

### Subtask:
Modify the `minWeekly_default` dictionary to reflect the increased weekly minimum hours for part-time baristas (Max, Jiwa, Fore) from 12 to 16 hours.


**Reasoning**:
To update the `minWeekly_default` dictionary, I will iterate through the part-time baristas ('Max', 'Jiwa', 'Fore') and set their minimum weekly hours to 16, while keeping the full-time baristas' values unchanged.



In [304]:
minWeekly_default_updated = minWeekly_default.copy()

for barista in ["Max", "Jiwa", "Fore"]:
    minWeekly_default_updated[barista] = 16

minWeekly_default = minWeekly_default_updated

print("Updated minWeekly_default dictionary:")
print(minWeekly_default)

Updated minWeekly_default dictionary:
{'Max': 16, 'Jiwa': 16, 'Fore': 16, 'Donna': 36, 'Paul': 36}


**Reasoning**:
Now that `minWeekly_default` has been updated, I will re-initialize the PuLP model, set `block_ids` back to the original 4 blocks, rebuild the decision variables and all constraints, and set the objective function.



In [305]:
model = pl.LpProblem("Barista_Scheduling_IncreasedMinWeekly", pl.LpMinimize)

# Set block_ids back to original 4 blocks
block_ids = list(blocks.keys())

# Rebuild decision variables with original 4 blocks
y = pl.LpVariable.dicts(
    "y",
    ((o, d, b) for o in baristas for d in days for b in block_ids),
    cat="Binary"
)

# Objective function
model += pl.lpSum(H * cost[o] * y[o, d, b] for o in baristas for d in days for b in block_ids)

# Constraints
# Coverage
for d in days:
    for b in block_ids:
        model += pl.lpSum(y[o, d, b] for o in baristas) >= 1

# Per-day availability (hours)
for o in baristas:
    for d in days:
        model += H * pl.lpSum(y[o, d, b] for b in block_ids) <= avail[(o, d)]

# Daily block limit
for o in baristas:
    for d in days:
        if etype[o] == "P":  # Part-time
            model += pl.lpSum(y[o, d, b] for b in block_ids) <= 2
        if etype[o] == "F":  # Full-time
            model += pl.lpSum(y[o, d, b] for b in block_ids) <= 3

# Weekly minimum hours (using the updated minWeekly_default)
for o in baristas:
    model += H * pl.lpSum(y[o, d, b] for d in days for b in block_ids) >= minWeekly_default[o]


**Reasoning**:
With the model rebuilt using the updated `minWeekly_default` and original 4 blocks, the next step is to solve this new model and then report its status, the total optimized cost, and the weekly hours for each barista as specified in the task.



In [306]:
model.solve()
print("Status:", pl.LpStatus[model.status])
print("Total Cost:", pl.value(model.objective))

print("\nWeekly Hours per Barista (with increased part-time minimums):")
for o in baristas:
    total_weekly_hours = sum(H * y[o, d, b].varValue for d in days for b in block_ids)
    print(f"  {o}: {total_weekly_hours} hours")

Status: Optimal
Total Cost: 13200000.0

Weekly Hours per Barista (with increased part-time minimums):
  Max: 16.0 hours
  Jiwa: 16.0 hours
  Fore: 16.0 hours
  Donna: 36.0 hours
  Paul: 36.0 hours


## Summary:

### Q&A
The solution status is "Optimal". The total optimized cost is \$13,200,000.0. The weekly hours for each barista are: Max: 16.0 hours, Jiwa: 16.0 hours, Fore: 16.0 hours, Donna: 36.0 hours, and Paul: 36.0 hours.

### Data Analysis Key Findings
*   The minimum weekly hours for part-time baristas Max, Jiwa, and Fore were increased from 12 to 16 hours.
*   The optimization model, rebuilt with the updated minimum weekly hours, achieved an "Optimal" solution.
*   The total optimized cost for the schedule is \$13,200,000.0.
*   The updated minimums directly influenced the scheduled hours for part-time staff, with Max, Jiwa, and Fore each scheduled for exactly 16.0 hours weekly. Full-time staff (Donna and Paul) maintained their 36.0 weekly hours.

### Insights or Next Steps
*   The adjustment of part-time minimum hours directly leads to an increase in their scheduled work, indicating that the baristas were previously scheduled at their lower minimum.
*   Consider evaluating the operational impact and additional labor cost associated with the increased part-time minimums, especially if there's flexibility to assign more hours to full-time staff or if higher part-time hours strain the budget.


# **Fairness variant**